<a href="https://colab.research.google.com/github/Adasyam31494/Recommendation-System-using-AI-HCI-Project/blob/main/Recommendation_System_using_AI_HCI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import files
uploaded = files.upload()

import zipfile
with zipfile.ZipFile("recommendation_data_large.zip", "r") as zip_ref:
    zip_ref.extractall("recommendation_data")

import pandas as pd
df = pd.read_csv("recommendation_data/product_ratings_large.csv")
print(df.head())

Saving recommendation_data_large (1).zip to recommendation_data_large (1).zip
   userId productId  rating
0      39       P16       1
1      29        P7       1
2      15        P4       1
3      43        P1       1
4       8        P5       2


In [10]:
# Create pivot table (user-item matrix)
user_item_matrix = df.pivot_table(index='userId', columns='productId', values='rating').fillna(0)
user_item_matrix.head()

productId,P1,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P2,P20,P3,P4,P5,P6,P7,P8,P9
userId,,,,,,,,,,,,,,,,,,,,
1,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,1.000000,1.0,0.0,0.000000,0.0,3.0,2.0,2.0,2.0,0.0,5.0
2,1.0,4.0,0.0,0.0,0.0,0.0,3.0,5.0,1.0,3.666667,0.0,0.0,2.333333,0.0,0.0,5.0,1.0,5.0,0.0,0.0
3,0.0,0.0,0.0,0.0,5.0,1.5,2.0,0.0,5.0,3.000000,0.0,4.0,3.000000,0.0,3.0,0.0,5.0,0.0,2.5,0.0
4,2.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,3.0,0.000000,0.0,0.0,0.000000,0.0,4.0,5.0,5.0,0.0,0.0,0.0
5,0.0,3.0,1.5,5.0,0.0,5.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity matrix
cosine_sim = cosine_similarity(user_item_matrix)

# Create a DataFrame for similarities
similarity_df = pd.DataFrame(cosine_sim, index=user_item_matrix.index, columns=user_item_matrix.index)
similarity_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.300677,0.339528,0.500255,0.367366,0.474414,0.330406,0.563976,0.238678,0.314485,...,0.631843,0.445455,0.095917,0.457902,0.585107,0.125073,0.384272,0.173119,0.398895,0.191834
2,0.300677,1.000000,0.269582,0.321885,0.324249,0.352452,0.111712,0.252201,0.486930,0.090577,...,0.440510,0.394830,0.013813,0.299861,0.393214,0.088420,0.136718,0.229361,0.218288,0.000000
3,0.339528,0.269582,1.000000,0.635499,0.192969,0.566474,0.512827,0.516394,0.304500,0.437688,...,0.435499,0.488458,0.407156,0.370299,0.271443,0.560193,0.268980,0.351771,0.574596,0.280337
4,0.500255,0.321885,0.635499,1.000000,0.172172,0.588341,0.410465,0.464796,0.468778,0.304604,...,0.471135,0.305071,0.510968,0.317463,0.113344,0.297350,0.240834,0.178857,0.342895,0.185807
5,0.367366,0.324249,0.192969,0.172172,1.000000,0.000000,0.255612,0.208065,0.000000,0.000000,...,0.529960,0.354217,0.430986,0.566570,0.336521,0.000000,0.300642,0.301816,0.225837,0.000000


In [12]:
def recommend_products(user_id, top_n=5):
    if user_id not in similarity_df.index:
        return f"User {user_id} not found."

    # Get the most similar users
    similar_users = similarity_df[user_id].sort_values(ascending=False)[1:]
    recommended_items = pd.Series(dtype='float64')

    for similar_user, similarity_score in similar_users.items():
        user_ratings = df[df['userId'] == similar_user]
        user_ratings = user_ratings.set_index('productId')['rating']
        recommended_items = recommended_items.add(user_ratings * similarity_score, fill_value=0)

    # Remove products already rated by the target user
    user_products = df[df['userId'] == user_id]['productId'].tolist()
    recommended_items = recommended_items.drop(labels=user_products, errors='ignore')

    return recommended_items.sort_values(ascending=False).head(top_n)

In [8]:
# Example: Get top 5 product recommendations for user with ID 10
recommend_products(10, top_n=5)

,0
productId,
P5,15.044533
P5,15.044533
P5,15.044533
P5,15.044533
P5,15.044533
